In [1]:
 # Python extension for interfacing with SQL and better table formatting with Pandas

# !pip install ipython-sql
# !pip install pandas

Our data for this project is initially in 2 CSV files. Let's investigate how these files look like first.

In [10]:
# load the extensions that we need to begin a separate cell
import pandas as pd
import sqlite3

In [11]:
# Getting a view of the census csv table
census_df = pd.read_csv('census_data.csv')
print(census_df.head())
print(census_df.info())

   zip_code state_code  pop_total  median_household_income  pct_edu_hs  \
0       601         PR      17800                  11507.0     24.1861   
1       602         PR      39716                  15511.0     18.3838   
2       603         PR      51565                  16681.0     23.6281   
3       606         PR       6320                  11648.0     32.7551   
4       610         PR      27976                  17751.0     28.7571   

   pct_edu_somecollege_under1yr  pct_edu_somecollege_1plusyrs  \
0                        1.2198                       12.9974   
1                        1.0237                        8.3569   
2                        1.6381                       10.2082   
3                        0.7076                        4.0402   
4                        1.3412                        9.2270   

   pct_edu_attain_assoc  pct_edu_attain_bach  pct_edu_attain_master  \
0                5.0812              15.5212                 2.3387   
1               13.392

Seems like this table contains information on different areas and some info on the people within them, mainly percentages of education and race. \
The zip_code should be the primary key for this table as well (zip_codes are unique), we will check this after we make our SQL database later however.

In [12]:
# Getting a view of the high school csv table
public_hs_df = pd.read_csv('public_hs_data.csv')
print(public_hs_df.head())
print(public_hs_df.info())

     school_id                school_name        street_address          city  \
0  10000500871       Albertville High Sch      402 E McCord Ave   Albertville   
1  10000600878           Douglas High Sch       225 Eagle Drive       Douglas   
2  10000600883  Kate D Smith DAR High Sch          6077 Main St         Grant   
3  10000601585       Brindlee Mt High Sch   994 Scant City Road  Guntersville   
4  10000700251            Hoover High Sch  1000 Buccaneer Drive        Hoover   

  state_code  zip_code  locale_code  pct_proficient_math  \
0         AL     35950         32.0            16.999999   
1         AL     35964         42.0            12.000000   
2         AL     35747         42.0            22.000000   
3         AL     35976         41.0            21.999999   
4         AL     35244         13.0            39.999998   

   pct_proficient_reading  pct_white  pct_black  pct_asian  pct_hispanic  \
0               54.000001    64.9018     2.0495     0.5978       31.4261   


Seems like this table contains information on the different public high schools including; location information, race percentages, proficiency percentages for math and reading.
One interesting thing to note is that locale code is actually related to the urbanization of the town, not really relating to the school itself directly. This is mentioned in the project instructions which gives the following table:

| locale_text | locale_code (locale_size)         |
|-------------|----------------------------------|
| City        | 11 (Large), 12 (Midsize), 13 (Small) |
| Suburb      | 21 (Large), 22 (Midsize), 23 (Small) |
| Town        | 31 (Fringe), 32 (Distant), 33 (Remote) |
| Rural       | 41 (Fringe), 42 (Distant), 43 (Remote) | \

The school_id should be the primary key of this table as it seems like it will be unique values. The zip_code would then be the foreign key to the previous table assuming we were correct earlier.

In [13]:
# Combining tables and making SQL database

# Make the SQLite database
conn = sqlite3.connect('education.db')

# Write DataFrames to SQLite tables in the database (don't need the dataframe indexes)
public_hs_df.to_sql('highschool', conn, if_exists='replace', index=False)
census_df.to_sql('census', conn, if_exists='replace', index=False)

# Close the connection
conn.close()

In [1]:
# Necessary in the Jupyter Notebook to load the SQL extension and connect to the database file to use SQL directly, currently using SQLite
# Formatting the SQL query outputs into a better format with Pandas

%load_ext sql
%sql sqlite:///education.db
%config SqlMagic.autopandas=True

Check our unique columns from earlier to make sure they can correctly be assumed to be the primary/foreign keys.

In [10]:
%%sql
SELECT
    COUNT(zip_code) AS total_values_census,
    COUNT(DISTINCT zip_code) AS unique_values_census
FROM
    census;

 * sqlite:///education.db
Done.


,total_values_census,unique_values_census
0,33120,33120


In [11]:
%%sql
SELECT
    COUNT(school_id) AS total_values_highschool,
    COUNT(DISTINCT school_id) AS unique_values_highschool
FROM
    highschool;

 * sqlite:///education.db
Done.


,total_values_highschool,unique_values_highschool
0,16623,16623


Looks like these columns can work as our keys. We won't be adjusting the tables since SQLite does not directly support this function. We could create a temp table and move around the data but this is unecessary to do. 

In [23]:
%%sql
PRAGMA table_info(census);

 * sqlite:///education.db
Done.


,cid,name,type,notnull,dflt_value,pk
0,0,zip_code,INTEGER,0,None,0
1,1,state_code,TEXT,0,None,0
2,2,pop_total,INTEGER,0,None,0
3,3,median_household_income,REAL,0,None,0
4,4,pct_edu_hs,REAL,0,None,0
5,5,pct_edu_somecollege_under1yr,REAL,0,None,0
6,6,pct_edu_somecollege_1plusyrs,REAL,0,None,0
7,7,pct_edu_attain_assoc,REAL,0,None,0
8,8,pct_edu_attain_bach,REAL,0,None,0
9,9,pct_edu_attain_master,REAL,0,None,0


In [24]:
%%sql
PRAGMA table_info(highschool);

 * sqlite:///education.db
Done.


,cid,name,type,notnull,dflt_value,pk
0,0,school_id,INTEGER,0,None,0
1,1,school_name,TEXT,0,None,0
2,2,street_address,TEXT,0,None,0
3,3,city,TEXT,0,None,0
4,4,state_code,TEXT,0,None,0
5,5,zip_code,INTEGER,0,None,0
6,6,locale_code,REAL,0,None,0
7,7,pct_proficient_math,REAL,0,None,0
8,8,pct_proficient_reading,REAL,0,None,0
9,9,pct_white,REAL,0,None,0


There is also no reason to change any of the other constraints about the columns as we will likely not be adding to these columns.

1. How many public high schools are in each zip code? in each state?

(There are too many ZIP codes to show them all so calculating the average number of schools per ZIP code will do instead.)

In [9]:
%%sql
WITH zip_school_count AS (
    SELECT zip_code, COUNT(*) AS "SchoolCount"
    FROM highschool
    GROUP BY zip_code
)
SELECT AVG(SchoolCount) AS "Avg School Count Per ZIP"
FROM zip_school_count;

 * sqlite:///education.db
Done.


,Avg School Count Per ZIP
0,1.242005


We could just use the high school table but we don't know for certain that each zip code has a high school in it. However, we also don't know if it is just that one of the tables got data in certain zip_codes that the other did not get. While originally I assumed that it meant that zip_code absolutely does not have a high school in it, we don't know for certain. Therefore, we will only work within the high school table for this question.

The query below was done with previously assumption that if there is a ZIP code in the census table and not in the high school table then it means that zip_code does not have a high school in it. This assumption is wrong, especially because the inverse could be true, a ZIP code could be in the high school table and not in the census table.

In [17]:
%%sql
WITH zip_school_count AS (
    SELECT census.zip_code AS "ZipCode", COUNT(*) AS "SchoolCount"
    FROM census
    LEFT JOIN highschool ON census.zip_code = highschool.zip_code
    GROUP BY census.zip_code
)
SELECT AVG(SchoolCount) AS "Avg School Count Per ZIP"
FROM zip_school_count;

 * sqlite:///education.db
Done.


,Avg School Count Per ZIP
0,1.097252


Further proof that the two tables do not cover the exact same areas can be seen based on the state codes. A few state codes have high school data but no census data! Therefore, for this question we can't just assume that the same population was used for both tables' data collection and will stick with just the high school table for the moment.

In [36]:
%%sql
SELECT DISTINCT state_code
FROM highschool
EXCEPT
SELECT DISTINCT state_code
FROM census;

 * sqlite:///education.db
Done.


,state_code
0,BI
1,GU
2,VI


In [38]:
%%sql
SELECT state_code, COUNT(*) AS "SchoolCount"
FROM highschool
GROUP BY state_code

 * sqlite:///education.db
Done.


,state_code,SchoolCount
0,AK,50
1,AL,275
2,AR,271
3,AZ,418
4,BI,18
5,CA,1294
6,CO,283
7,CT,170
8,DC,28
9,DE,28


2. The locale_code column in the high school data corresponds to various levels of urbanization as listed below. Use the CASE statement to display the corresponding locale_text and locale_size in your query result.
   | locale_text | locale_code (locale_size)              |
   | ----------- | -------------------------------------- |
   | City        | 11 (Large), 12 (Midsize), 13 (Small)   |
   | Suburb      | 21 (Large), 22 (Midsize), 23 (Small)   |
   | Town        | 31 (Fringe), 32 (Distant), 33 (Remote) |
   | Rural       | 41 (Fringe), 42 (Distant), 43 (Remote) |

Let's just look at how the locale_code looks again

In [12]:
%%sql
SELECT locale_code FROM highschool LIMIT 5;

 * sqlite:///education.db
Done.


,locale_code
0,32.0
1,42.0
2,42.0
3,41.0
4,13.0


In case we need the locale_text and locale_size parameters later, make a temp table that includes them.

In [78]:
%%sql
CREATE TEMPORARY TABLE temp_locale AS
    SELECT 
        school_id,
        school_name,
        locale_code,
        CASE
            WHEN locale_code > 10 AND locale_code < 14 THEN 'City'
            WHEN locale_code > 20 AND locale_code < 24 THEN 'Suburb'
            WHEN locale_code > 30 AND locale_code < 34 THEN 'Town'
            WHEN locale_code > 40 AND locale_code < 44 THEN 'Rural'
            ELSE 'Unknown'
        END AS locale_text,
        CASE
            WHEN locale_code = 11 OR locale_code = 21 THEN 'Large'
            WHEN locale_code = 12 OR locale_code = 22 THEN 'Midsize'
            WHEN locale_code = 13 OR locale_code = 23 THEN 'Small'
            WHEN locale_code = 31 OR locale_code = 41 THEN 'Fringe'
            WHEN locale_code = 32 OR locale_code = 42 THEN 'Distant'
            WHEN locale_code = 33 OR locale_code = 43 THEN 'Remote'
            ELSE 'Unknown'
        END AS locale_size
    FROM highschool;

 * sqlite:///education.db
Done.


""


Checking the first few rows in the table to make sure it looks right.

In [79]:
%%sql
SELECT school_id, school_name, locale_code, locale_text, locale_size 
FROM temp_locale
ORDER BY school_name 
LIMIT 5;

 * sqlite:///education.db
Done.


,school_id,school_name,locale_code,locale_text,locale_size
0,251251002790,21st Century Skills Academy,21.0,Suburb,Large
1,270002504190,287 OLL ACADEMIC,13.0,City,Small
2,480735000001,A & M CONS H S,13.0,City,Small
3,170654000339,A B Shepard High Sch (Campus),21.0,Suburb,Large
4,480972000442,A C JONES H S,32.0,Town,Distant


Let's now take a look at the number of schools for each different value of the locale information and what the locale text and sizes would be for each.

In [80]:
%%sql
SELECT locale_code, locale_text, locale_size, COUNT(*)
FROM temp_locale
GROUP BY locale_code;

 * sqlite:///education.db
Done.


,locale_code,locale_text,locale_size,COUNT(*)
0,NaN,Unknown,Unknown,7
1,11.0,City,Large,2024
2,12.0,City,Midsize,745
3,13.0,City,Small,879
4,21.0,Suburb,Large,3554
5,22.0,Suburb,Midsize,464
6,23.0,Suburb,Small,311
7,31.0,Town,Fringe,549
8,32.0,Town,Distant,1219
9,33.0,Town,Remote,764


Some seems to not have an actualy locale_code but we'll keep these in our dataset for now.

3. What is the minimum, maximum, and average median_household_income of the nation? for each state?

In [84]:
%%sql
SELECT MIN(median_household_income) AS "Min Median Household Income", MAX(median_household_income) AS "Max Median Household Income", AVG(median_household_income) AS "Avg Median Household Income"
FROM census;

 * sqlite:///education.db
Done.


,Min Median Household Income,Max Median Household Income,Avg Median Household Income
0,2499.0,250001.0,54683.115936


In [85]:
%%sql
SELECT state_code, MIN(median_household_income) AS "Min Median Household Income", MAX(median_household_income) AS "Max Median Household Income", AVG(median_household_income) AS "Avg Median Household Income"
FROM census
WHERE median_household_income IS NOT NULL
GROUP BY state_code;

 * sqlite:///education.db
Done.


,state_code,Min Median Household Income,Max Median Household Income,Avg Median Household Income
0,AK,16667.0,170329.0,52698.246575
1,AL,2499.0,203083.0,42463.270451
2,AR,14479.0,99205.0,39647.679702
3,AZ,11384.0,129564.0,48602.519789
4,CA,2499.0,250001.0,65724.770129
5,CO,13750.0,194750.0,58976.198745
6,CT,11755.0,218152.0,84020.728302
7,DC,30665.0,165425.0,81712.130435
8,DE,26810.0,140400.0,62800.413793
9,FL,2499.0,232500.0,50791.418605


4. Do characteristics of the zip-code area, such as median household income, influence students’ performance in high school?

The project page also gives a hint that an option to do this is to 'use the CASE statement to divide the median_household_income into income ranges (e.g., <$50k, $50k-$100k, $100k+) and find the average exam scores for each.'

So let's start with that approach first.

Also, since we need data in both tables, we can only use ZIP codes found in both for this question.

In [77]:
%%sql
WITH IncomeClass AS (
    SELECT zip_code,
    CASE
        WHEN median_household_income < 50000 THEN '<$50K'
        WHEN median_household_income >= 50000 AND median_household_income < 100000 THEN '$50K-$100K'
        WHEN median_household_income >= 100000 THEN '$100K+'
        ELSE "Unknown"
    END AS income_class
    FROM census
    WHERE median_household_income IS NOT NULL
)

SELECT COUNT(*) AS 'Number High Schools', IncomeClass.income_class, AVG(highschool.pct_proficient_math) AS 'Avg % Proficient Math', AVG(highschool.pct_proficient_reading) AS 'Avg % Proficient Reading'
FROM IncomeClass
JOIN highschool ON IncomeClass.zip_code = highschool.zip_code
GROUP BY income_class
ORDER BY income_class; 

 * sqlite:///education.db
Done.


,Number High Schools,income_class,Avg % Proficient Math,Avg % Proficient Reading
0,808,$100K+,67.939995,77.067588
1,7517,$50K-$100K,52.307381,64.115713
2,8115,<$50K,43.713075,55.157304


It seems like the median household income does indeed correlate to the profiency in high school.

Maybe the locale (the size of the place) also correlates. Let's use that temp table from earlier and use the locale information we made.

In [131]:
%%sql
SELECT COUNT(*) AS 'Number High Schools', temp_locale.locale_text, AVG(highschool.pct_proficient_math) AS 'Avg % Proficient Math', AVG(highschool.pct_proficient_reading) AS 'Avg % Proficient Reading'
FROM temp_locale
JOIN highschool ON temp_locale.school_id = highschool.school_id
GROUP BY temp_locale.locale_text; 

 * sqlite:///education.db
Done.


,Number High Schools,locale_text,Avg % Proficient Math,Avg % Proficient Reading
0,3648,City,45.859304,56.592991
1,6107,Rural,50.451240,61.438421
2,4329,Suburb,50.420814,63.129770
3,2532,Town,46.233965,58.077341
4,7,Unknown,7.500000,32.000000


Remember that the scale goes; City -> Suburb -> Town -> Rural in terms of biggest to smallest.

Not nearly as much of a correlation compared to the median income.

Let's now look at some of the percentages for levels of education in the ZIP codes. We could look at them all but let's just pick some, specifically High School Diploma, Bachelor Degree, and Doctorate. No real reason for these specific ones other than it covers all the ranges of the education levels available.

In [125]:
%%sql
CREATE TEMPORARY TABLE temp_edu_buckets AS
    SELECT school_id, pct_proficient_math, pct_proficient_reading, pct_edu_hs, pct_edu_attain_bach, pct_edu_attain_doct,
       NTILE(3) OVER (ORDER BY pct_edu_hs) AS hs_bucket,
       NTILE(3) OVER (ORDER BY pct_edu_attain_bach) AS bach_bucket,
       NTILE(3) OVER (ORDER BY pct_edu_attain_doct) AS doct_bucket
    FROM census
    JOIN highschool ON census.zip_code = highschool.zip_code
    WHERE pct_edu_hs IS NOT NULL;

 * sqlite:///education.db
Done.


""


In [128]:
%%sql
SELECT AVG(pct_edu_hs) AS 'Average % With HS Diploma', hs_bucket,
    AVG(pct_proficient_math) AS 'Avg % Proficient Math', 
    AVG(pct_proficient_reading) AS 'Avg % Proficient Reading'
FROM temp_edu_buckets
GROUP BY hs_bucket;

 * sqlite:///education.db
Done.


,Average % With HS Degree,hs_bucket,Avg % Proficient Math,Avg % Proficient Reading
0,17.074694,1,50.825532,63.455719
1,26.462848,2,44.755532,56.757229
2,35.712750,3,51.025390,60.911076


In [129]:
%%sql
SELECT AVG(pct_edu_attain_bach)  AS 'Average % With Bachelor Degree', bach_bucket,
    AVG(pct_proficient_math) AS 'Avg % Proficient Math', 
    AVG(pct_proficient_reading) AS 'Avg % Proficient Reading'
FROM temp_edu_buckets
GROUP BY bach_bucket;

 * sqlite:///education.db
Done.


,Average % With Bachelor Degree,bach_bucket,Avg % Proficient Math,Avg % Proficient Reading
0,8.308219,1,44.359304,55.325377
1,14.486718,2,47.697543,59.485779
2,25.956835,3,54.553608,66.305389


In [130]:
%%sql
SELECT AVG(pct_edu_attain_doct)  AS 'Average % With Doctorate', doct_bucket,
    AVG(pct_proficient_math) AS 'Avg % Proficient Math', 
    AVG(pct_proficient_reading) AS 'Avg % Proficient Reading'
FROM temp_edu_buckets
GROUP BY doct_bucket;

 * sqlite:///education.db
Done.


,Average % With Doctorate,doct_bucket,Avg % Proficient Math,Avg % Proficient Reading
0,0.140630,1,46.241308,57.321833
1,0.648191,2,46.781736,58.507927
2,2.331711,3,53.572764,65.262255


Surprisingly, it doesn't seem that the % of people with a high school diploma has much correlation to the high school grades while % of people with higher levels educations (bachelor and doctorate) seem to correlate with high school grades.

5. On average, do students perform better on the math or reading exam? Find the number of states where students do better on the math exam, and vice versa

Sometimes we don't have specific exam due to some states not having standardized tests (according to the project description)

In [154]:
%%sql
SELECT COUNT(*)
FROM highschool
WHERE pct_proficient_math IS NULL OR
pct_proficient_reading IS NULL;

 * sqlite:///education.db
Done.


,COUNT(*)
0,895


In [132]:
%%sql
SELECT AVG(pct_proficient_math), AVG(pct_proficient_reading)
FROM highschool
WHERE pct_proficient_math IS NOT NULL AND
pct_proficient_reading IS NOT NULL;

 * sqlite:///education.db
Done.


,AVG(pct_proficient_math),AVG(pct_proficient_reading)
0,48.849262,60.337332


Students are better on average at the reading exam.

Let's look at which exams have better avg scores for the different subjects and then get the total numbers of states for each. 

In [139]:
%%sql
WITH state_avg AS(
    SELECT state_code, AVG(pct_proficient_math) AS 'avg_math', AVG(pct_proficient_reading) AS 'avg_read'
    FROM highschool
    GROUP BY state_code
)

SELECT state_code, 
    CASE 
        WHEN avg_math IS NULL OR avg_read IS NULL THEN 'No Exam Data'
        WHEN avg_math > avg_read THEN 'Math Better'
        WHEN avg_math < avg_read THEN 'Reading Better'
        ELSE 'Error'
    END AS best_subject,
    avg_math, avg_read
FROM state_avg;

 * sqlite:///education.db
Done.


,state_code,best_subject,avg_math,avg_read
0,AK,Reading Better,24.649436,33.579949
1,AL,Reading Better,17.145938,54.510687
2,AR,Reading Better,17.878249,33.760448
3,AZ,Reading Better,35.360354,42.016889
4,BI,No Exam Data,NaN,NaN
5,CA,Reading Better,29.365608,56.024747
6,CO,Reading Better,22.961019,37.806369
7,CT,Reading Better,30.683944,52.227272
8,DC,Reading Better,9.276794,24.678631
9,DE,Reading Better,21.535405,46.267857


In [141]:
%%sql
WITH state_avg AS(
    SELECT state_code, AVG(pct_proficient_math) AS 'avg_math', AVG(pct_proficient_reading) AS 'avg_read'
    FROM highschool
    GROUP BY state_code
)

SELECT COUNT(*) AS 'Number of States', 
    CASE 
        WHEN avg_math IS NULL OR avg_read IS NULL THEN 'No Exam Data'
        WHEN avg_math > avg_read THEN 'Math Better'
        WHEN avg_math < avg_read THEN 'Reading Better'
        ELSE 'Error'
    END AS best_subject
FROM state_avg
GROUP BY best_subject;

 * sqlite:///education.db
Done.


,Number of States,best_subject
0,7,Math Better
1,3,No Exam Data
2,45,Reading Better


Most states have better reading scores, which makes sense because it was had higher overall scores.

6. What is the average proficiency on state assessment exams for each zip code, and how do they compare to other zip codes in the same state?

In [153]:
%%sql
WITH zip_avgs AS (
    SELECT zip_code, state_code, ROUND(AVG(pct_proficient_math), 2) AS 'avg_math', ROUND(AVG(pct_proficient_reading), 2) AS 'avg_read' 
    FROM highschool
    WHERE pct_proficient_math IS NOT NULL 
    GROUP BY zip_code
),
state_avg AS(
    SELECT state_code, AVG(pct_proficient_math) AS 'avg_math', AVG(pct_proficient_reading) AS 'avg_read'
    FROM highschool
    WHERE pct_proficient_math IS NOT NULL
    GROUP BY state_code
)
SELECT 
    zip_avgs.zip_code, 
    state_avg.state_code, 
    zip_avgs.avg_math AS 'Zip Avg Math', 
    state_avg.avg_math AS 'State Avg Math', 
    ROUND(((zip_avgs.avg_math - state_avg.avg_math) / state_avg.avg_math) * 100, 2) AS 'Zip Average Math Compared to State Avg (%)',
    zip_avgs.avg_read AS 'Zip Avg Read', 
    state_avg.avg_read AS 'State Avg Read',
    ((zip_avgs.avg_read - state_avg.avg_read) / state_avg.avg_read) * 100 AS 'Zip Average Read Compared to State Avg (%)'
FROM zip_avgs
JOIN state_avg ON zip_avgs.state_code = state_avg.state_code;

 * sqlite:///education.db
Done.


,zip_code,state_code,Zip Avg Math,State Avg Math,Zip Average Math Compared to State Avg (%),Zip Avg Read,State Avg Read,Zip Average Read Compared to State Avg (%)
0,602,PR,23.75,7.740366,206.83,60.50,36.875162,64.067074
1,603,PR,2.50,7.740366,-67.70,22.00,36.875162,-40.339246
2,605,PR,7.50,7.740366,-3.11,57.00,36.875162,54.575590
3,606,PR,2.51,7.740366,-67.57,52.00,36.875162,41.016328
4,610,PR,30.75,7.740366,297.27,68.50,36.875162,85.761893
...,...,...,...,...,...,...,...,...
12998,99835,AK,27.00,24.649436,9.54,32.00,33.579949,-4.705037
12999,99901,AK,32.00,24.649436,29.82,41.00,33.579949,22.096672
13000,99921,AK,24.49,24.649436,-0.65,44.51,33.579949,32.549338
13001,99926,AK,15.00,24.649436,-39.15,15.00,33.579949,-55.330486
